# MLOps End-to-End: From Training to Deployment

This notebook demonstrates a complete MLOps workflow using the AI-Mastery-2026 toolkit.

**What you'll learn:**
1. Train a model from scratch using toolkit components
2. Save the model for production use
3. Serve the model via API
4. Make predictions and monitor performance

## Part 1: Data Preparation

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generate synthetic dataset
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=8,
    n_redundant=2,
    n_classes=2,
    random_state=42
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Standardize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Training samples: {X_train.shape[0]}")
print(f"Test samples: {X_test.shape[0]}")
print(f"Features: {X_train.shape[1]}")

ModuleNotFoundError: No module named 'numpy'

## Part 2: Train From-Scratch Neural Network

In [ ]:
import sys
sys.path.insert(0, '..')

from src.ml.deep_learning import (
    NeuralNetwork, Dense, Activation, Dropout, 
    BatchNormalization, CrossEntropyLoss
)

# Build model
model = NeuralNetwork()
model.add(Dense(10, 64, weight_init='he'))
model.add(BatchNormalization(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(64, 32, weight_init='he'))
model.add(Activation('relu'))
model.add(Dense(32, 2, weight_init='xavier'))
model.add(Activation('softmax'))

# Compile
model.compile(loss=CrossEntropyLoss(), learning_rate=0.01)

# Summary
model.summary()

In [ ]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=True
)

In [ ]:
import matplotlib.pyplot as plt

# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['loss'], label='Train Loss')
axes[0].plot(history['val_loss'], label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].set_title('Training Loss')

axes[1].plot(history['accuracy'], label='Train Acc')
axes[1].plot(history['val_accuracy'], label='Val Acc')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].set_title('Training Accuracy')

plt.tight_layout()
plt.show()

## Part 3: Evaluate Model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Evaluate
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# Get predictions
y_pred = model.predict(X_test)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## Part 4: Save Model for Production

In [ ]:
import pickle
import joblib
import os

# Create models directory
os.makedirs('../models', exist_ok=True)

# Save neural network parameters
model_data = {
    'layer_params': [layer.get_params() for layer in model.layers],
    'layer_types': [layer.__class__.__name__ for layer in model.layers],
    'scaler_mean': scaler.mean_,
    'scaler_scale': scaler.scale_,
    'metadata': {
        'model_type': 'NeuralNetwork',
        'n_features': 10,
        'n_classes': 2,
        'test_accuracy': float(test_acc),
        'test_loss': float(test_loss)
    }
}

# Save with joblib
model_path = '../models/mlops_demo_model.joblib'
joblib.dump(model_data, model_path)
print(f"Model saved to: {model_path}")

## Part 5: Start Docker Services

Run these commands in terminal:

```bash
# Start all services
docker-compose up -d

# Check status
docker-compose ps

# View API logs
docker-compose logs -f api
```

## Part 6: Test API Endpoints

In [ ]:
import requests

API_URL = "http://localhost:8000"

# Health check
response = requests.get(f"{API_URL}/health")
print("Health Check:")
print(response.json())

In [ ]:
# List available models
response = requests.get(f"{API_URL}/models")
print("\nAvailable Models:")
print(response.json())

In [ ]:
# Make prediction
sample = X_test[0].tolist()

response = requests.post(
    f"{API_URL}/predict",
    json={"features": sample, "model_name": "classification_model"}
)

print("\nPrediction Result:")
result = response.json()
print(f"  Prediction: {result['predictions']}")
print(f"  Actual: {y_test[0]}")
print(f"  Processing time: {result['processing_time']*1000:.2f}ms")

In [ ]:
# Batch prediction
batch_samples = X_test[:10].tolist()

response = requests.post(
    f"{API_URL}/predict/batch",
    json={"features": batch_samples, "model_id": "classification_model"}
)

print("\nBatch Prediction:")
result = response.json()
print(f"  Predictions: {result['predictions']}")
print(f"  Actual: {y_test[:10].tolist()}")
print(f"  Accuracy: {np.mean(np.array(result['predictions']) == y_test[:10]):.2%}")

## Part 7: View Metrics

Access monitoring dashboards:

- **Prometheus**: http://localhost:9090
- **Grafana**: http://localhost:3000 (admin/admin)

In [ ]:
# Get API metrics
response = requests.get(f"{API_URL}/metrics")
print("API Metrics:")
print(response.json())

## Summary

In this notebook, we completed the full MLOps cycle:

1. ✅ **Data Preparation** - Generated and preprocessed data
2. ✅ **Model Training** - Built neural network from scratch
3. ✅ **Evaluation** - Assessed model performance
4. ✅ **Model Saving** - Serialized for production
5. ✅ **API Deployment** - Served via FastAPI in Docker
6. ✅ **Predictions** - Made real-time predictions
7. ✅ **Monitoring** - Tracked metrics with Prometheus/Grafana

### Next Steps

- Add model versioning
- Implement A/B testing
- Set up CI/CD pipeline
- Add data validation
- Configure alerting